In [36]:
import numpy as np
import re 
from operator import itemgetter

In [37]:
def def_matriz(seq_vert, seq_horiz):

    num_col = len(seq_vert) + 1
    num_lin = len(seq_horiz) + 1
    matriz = np.zeros((num_col, num_lin))

    return matriz

In [38]:

def read_arq():
    with open('input.txt', 'r') as arquivo:
        linhas = arquivo.readlines()
    seq_vert = linhas[0].strip()
    seq_horiz = linhas[1].strip()
    GAP = linhas[2].strip()
    mismatch = linhas[3].strip()
    match = linhas[4].strip()
    return seq_vert, seq_horiz, GAP, mismatch, match


In [39]:
def comp_sequence(matriz, seq_vert, seq_horiz, mismatch, match, i, j):
    if seq_vert[i-1] == seq_horiz[j-1]:
        return int(matriz[i-1][j-1]) + int(match)
    else:
        return int(matriz[i-1][j-1]) + int(mismatch)
        

In [40]:
def smith_waterman(matriz, seq_vert, seq_horiz, GAP, mismatch, match):
    GAP = int(GAP)
    mismatch = int(mismatch)
    match = int(match)
    num_lin = len(matriz)
    num_col = len(matriz[0])

    sum = 0

    #set GAP first column
    for i in range(num_lin):
        for j in range(num_col):
            if j == 0 and i != 0:
                sum = sum + GAP
                matriz[i][j] = sum

    sum = 0
    
    #set GAP first line
    for i in range(num_lin):
        for j in range(num_col):
            if i == 0 and j != 0:
                sum = sum + GAP
                matriz[i][j] = sum
    

    return matriz
            

In [41]:
def backtrace(matriz, matriz_back, seq_vert, seq_horiz):
    #left seq1 é GAP
    #up seq2 é GAP

    
    return matriz

In [42]:
# teste de uso das variáveis
print("Vertical:", read_arq()[0])
print("Horizontal:", read_arq()[1])
print("GAP:", read_arq()[2])
print("Mismatch:", read_arq()[3])
print("Match:", read_arq()[4])

matriz_initial = def_matriz(read_arq()[0], read_arq()[1])
matriz = smith_waterman(matriz_initial, read_arq()[0], read_arq()[1], read_arq()[2], read_arq()[3], read_arq()[4])

print(matriz)


Vertical: ATCG
Horizontal: TCG
GAP: -2
Mismatch: -1
Match: 1
[[ 0. -2. -4. -6.]
 [-2.  0.  0.  0.]
 [-4.  0.  0.  0.]
 [-6.  0.  0.  0.]
 [-8.  0.  0.  0.]]
